In [94]:
%load_ext autoreload
%autoreload 2

from dataloader import FakenNewsNet
import tensorflow as tf
from model import Net, GNNCL
from spektral.data import DisjointLoader
from tqdm import tqdm
import numpy as np

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import binary_accuracy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
def train_step(model, loss_fn, optimizer, inputs, target):
    with tf.GradientTape() as tape:
        pred = model(inputs[:2])  # drop edge features from inputs
        loss = loss_fn(target, pred)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(binary_accuracy(target, pred))
    return loss, acc

In [85]:
def random_split(data, train_pct=0.75, train_epochs=5, batch_size=1, seed=None):
    """ returns two DataLoaders, resp. for train and test set 
        according to the input split percentage """
    if seed is not None:
        np.random.default_rng(seed)

    np.random.shuffle(data)
    n = len(data)
    train_size = int(n*train_pct)

    train_set = data[:train_size]
    test_set = data[train_size:]

    train_loader = DisjointLoader(train_set, batch_size=batch_size, epochs=train_epochs)
    test_loader = DisjointLoader(test_set, batch_size=len(test_set), epochs=1)

    return train_loader, test_loader

In [86]:
def evaluate(train_loader, test_loader, num_classes=1):
    model = GNNCL(num_classes=num_classes)
    loss_fn = BinaryCrossentropy()
    optimizer = Adam(learning_rate=0.001)

    epoch = step = 0
    train_losses = []
    train_accuracies = []

    for inputs, target in tqdm(train_loader):
        step += 1
        loss, acc = train_step(model, loss_fn, optimizer, inputs, target)
        train_losses.append(loss)
        train_accuracies.append(acc)

        if step == train_loader.steps_per_epoch:
            step = 0
            epoch += 1
            print("Ep. {} - Loss: {:.3f}. Acc: {:.3f}".format(epoch, np.mean(train_losses), np.mean(train_accuracies)))
            train_losses = []
            train_accuracies = []
    
    test_losses = []
    test_accuracies = []

    for inputs, target in test_loader:
        pred = model(inputs[:2], training=False)
        print(pred[:10], target[:10])
        test_losses.append(loss_fn(target, pred))
        test_accuracies.append(tf.reduce_mean(binary_accuracy(target, pred)))
        
    print("Done. Test loss: {:.3f}. Test acc: {:.3f}".format(np.mean(test_losses), np.mean(test_accuracies)))

    return train_losses, train_accuracies, test_losses, test_accuracies

In [96]:
data = FakenNewsNet("./dataset/FakeNewsNet/politifact/")  

crossval_results = []
n_splits = 1

for i in range(n_splits):
    print(f"Experiment {i+1} of {n_splits}")
    train_loader, test_loader = random_split(data, train_epochs=50, batch_size=1)

    train_loss, train_acc, test_loss, test_acc = evaluate(train_loader, test_loader)

    crossval_results.append((train_loss, train_acc, test_loss, test_acc))

Experiment 1 of 1


281it [00:27, 10.81it/s]